In [65]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.utils
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import random

# Split the Data

In [66]:
# Directories containing preprocessed images
save_directory_real = '../data/interim/face_processed/real'
save_directory_fake = '../data/interim/face_processed/fake'

# Function to create a DataFrame with image paths and labels
def create_image_label_df(real_dir, fake_dir):
    real_images = [(os.path.join(real_dir, f), 1) for f in os.listdir(real_dir) if os.path.isfile(os.path.join(real_dir, f))]
    fake_images = [(os.path.join(fake_dir, f), 0) for f in os.listdir(fake_dir) if os.path.isfile(os.path.join(fake_dir, f))]
    
    # Combine real and fake images into one DataFrame
    df = pd.DataFrame(real_images + fake_images, columns=['image_path', 'label'])
    return df

# Create the DataFrame
images_df = create_image_label_df(save_directory_real, save_directory_fake)

In [67]:
# Splitting the data into features and target arrays
X = images_df['image_path'].values
y = images_df['label'].values

# First split to separate out the test set
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Second split to separate out the training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)  # 0.25 x 0.8 = 0.2

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Testing set size: {len(X_test)}")

'''
Training Set: 60% of the original dataset.
Validation Set: 20% of the original dataset.
Testing Set: 20% of the original dataset.
'''

Training set size: 2400
Validation set size: 800
Testing set size: 800


'\nTraining Set: 60% of the original dataset.\nValidation Set: 20% of the original dataset.\nTesting Set: 20% of the original dataset.\n'

# Loading images for modelling

In [81]:
def load_images(image_paths, labels):
    images = []
    for image_path in image_paths:
        img = load_img(image_path, target_size=(128, 128), color_mode='grayscale') #Note: gray scale and size pre-processed alr
        img_array = img_to_array(img)/255.0 #Convert image to array + normalise 
        images.append(img_array) 
    return np.array(images), np.array(labels)

# Load and get images in dataset (From path)
X_train_processed, y_train_processed = load_images(X_train, y_train)
X_val_processed, y_val_processed = load_images(X_val, y_val)
X_test_processed, y_test_processed = load_images(X_test, y_test)

In [82]:
X_train_processed.shape

(2400, 128, 128, 1)

# Modelling

- Base Model

In [83]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=(128,128,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
#model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 14, 14, 128)     

In [84]:
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

In [87]:
model.fit(X_train_processed, y_train_processed,
          epochs=10, batch_size=32, 
          validation_data=(X_val_processed, y_val_processed))

Epoch 1/10
75/75 [==============================] - 33s 438ms/step - loss: 0.6919 - accuracy: 0.5204 - val_loss: 0.6907 - val_accuracy: 0.6012
Epoch 2/10
75/75 [==============================] - 35s 468ms/step - loss: 0.6913 - accuracy: 0.5333 - val_loss: 0.6905 - val_accuracy: 0.5925
Epoch 3/10
75/75 [==============================] - 36s 483ms/step - loss: 0.6912 - accuracy: 0.5304 - val_loss: 0.6903 - val_accuracy: 0.6012
Epoch 4/10
75/75 [==============================] - 31s 415ms/step - loss: 0.6901 - accuracy: 0.5633 - val_loss: 0.6901 - val_accuracy: 0.5600
Epoch 5/10
75/75 [==============================] - 32s 424ms/step - loss: 0.6914 - accuracy: 0.5346 - val_loss: 0.6898 - val_accuracy: 0.5663
Epoch 6/10
75/75 [==============================] - 39s 527ms/step - loss: 0.6902 - accuracy: 0.5492 - val_loss: 0.6896 - val_accuracy: 0.5288
Epoch 7/10
75/75 [==============================] - 37s 495ms/step - loss: 0.6896 - accuracy: 0.5458 - val_loss: 0.6889 - val_accuracy: 0.5813

In [88]:
loss, accuracy = model.evaluate(X_test_processed, y_test_processed)
print(accuracy)

25/25 [==============================] - 3s 132ms/step - loss: 0.6870 - accuracy: 0.5800
0.5799999833106995
